#DQNPrioritized experience replay

## libraries and data

In [ ]:
from google.colab import drive
import sys, os

#mount google drive
drive.mount('/content/drive', force_remount=False)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
!pip install gym pyvirtualdisplay > /dev/null 2>&1
!apt-get install -y xvfb python-opengl ffmpeg > /dev/null 2>&1
!pip install gym > /dev/null 2>&1
!pip install pyglet > /dev/null 2>&1

In [ ]:
os.chdir("/content/drive/My Drive/CS5446 Project")

In [ ]:
!python -m atari_py.import_roms Roms

copying adventure.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Adventure (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/adventure.bin
copying air_raid.bin from HC ROMS/BY ALPHABET (PAL)/A-G/Air Raid (PAL).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/air_raid.bin
copying alien.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Alien.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/alien.bin
copying crazy_climber.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Crazy Climber.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/crazy_climber.bin
copying elevator_action.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Elevator Action (Prototype).bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_roms/elevator_action.bin
copying gravitar.bin from HC ROMS/BY ALPHABET (PAL)/A-G/REMAINING NTSC ORIGINALS/Gravitar.bin to /usr/local/lib/python3.7/dist-packages/atari_py/atari_

## Algorithm

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
import torch.autograd as autograd 
import torch.nn.functional as F

import gym
from gym.wrappers import Monitor
import numpy as np
import glob
import io
import base64
from IPython.display import HTML
from pyvirtualdisplay import Display
from IPython import display as ipythondisplay

import math, random

### Environment and parameter

In [ ]:
seed = 2022
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
torch.cuda.manual_seed_all(seed)

env_id = "CartPole-v0"
env = gym.make(env_id)

epsilon_start = 1.0
epsilon_final = 0.01
epsilon_decay = 500
epsilon_by_frame = lambda frame_idx: epsilon_final + (epsilon_start - epsilon_final) * math.exp(-1. * frame_idx / epsilon_decay)

num_frames = 10000
batch_size = 32
gamma      = 0.99

### Replay Buffer

In [ ]:

from collections import deque,namedtuple

# Standard DQN
# class ReplayBuffer(object):
#     def __init__(self, capacity):
#         self.buffer = deque(maxlen=capacity)
    
#     def push(self, state, action, reward, next_state, done):
#         state      = np.expand_dims(state, 0)
#         next_state = np.expand_dims(next_state, 0)
            
#         self.buffer.append((state, action, reward, next_state, done))
    
#     def sample(self, batch_size):
#         state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
#         return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
#     def __len__(self):
#         return len(self.buffer)


# Pioritized

# update_mem_every = 100
# update_nn_every = 10

# experiences_per_sampling = math.ceil(batch_size * update_mem_every / update_nn_every)

# class ReplayBuffer(object):
#     def __init__(self, capacity):
#         self.experience_count = 0

#         self.priorities_max = 10
#         self.weights_max = 5

#         self.buffer_size = capacity
#         self.experience = namedtuple("Experience", 
#         field_names=["state", "action", "reward", "next_state", "done"])
#         self.data = namedtuple("Data", 
#             field_names=["priority", "probability", "weight","index"])

#         indexes = []
#         datas = []
#         for i in range(capacity):
#             indexes.append(i)
#             d = self.data(0,0,0,i)
#             datas.append(d)

#         self.memory = {key: self.experience for key in indexes}
#         self.memory_data = {key: data for key,data in zip(indexes, datas)}

        
    
#     # def push(self, state, action, reward, next_state, done):
#     #     state      = np.expand_dims(state, 0)
#     #     next_state = np.expand_dims(next_state, 0)
            
#     #     self.buffer.append((state, action, reward, next_state, done))
    
#     # def sample(self, batch_size):
#     #     state, action, reward, next_state, done = zip(*random.sample(self.buffer, batch_size))
#     #     return np.concatenate(state), action, reward, np.concatenate(next_state), done
    
#     def __len__(self):
#         return len(self.buffer)

#     def choices(self, population, weights=None, *, cum_weights=None, k=1):
#         """Return a k sized list of population elements chosen with replacement.
#         If the relative weights or cumulative weights are not specified,
#         the selections are made with equal probability.
#         """
        
#        ###  Removed the rest of the implementation for visibility ###
       
#         return [population[bisect(cum_weights, random() * total, 0, hi)]
#                 for i in _repeat(None, k)]

#     def push(self, state, action, reward, next_state, done):
#         """Add a new experience to memory."""
#         self.experience_count += 1
#         index = self.experience_count % self.buffer_size

#         if self.experience_count > self.buffer_size:
#             temp = self.memory_data[index]
#             self.priorities_sum_alpha -= temp.probability**self.alpha
#             if temp.priority == self.priorities_max:
#                 self.memory_data[index].priority = 0
#                 self.priorities_max = max(self.memory_data.items(), key=operator.itemgetter(1)).priority
#             if self.compute_weights:
#                 if temp.weight == self.weights_max:
#                     self.memory_data[index].weight = 0
#                     self.weights_max = max(self.memory_data.items(), key=operator.itemgetter(2)).weight

#         priority = self.priorities_max
#         weight = self.weights_max
#         self.priorities_sum_alpha += priority ** self.alpha
#         probability = priority ** self.alpha / self.priorities_sum_alpha
#         e = self.experience(state, action, reward, next_state, done)
#         self.memory[index] = e
#         d = self.data(priority, probability, weight, index)
#         self.memory_data[index] = d

#     def sample(self):
#             """Randomly sample X batches of experiences from memory."""
#             # X is the number of steps before updating memory
#             self.current_batch = 0
#             values = list(self.memory_data.values())
#             random_values = random.choices(self.memory_data, 
#                                           [data.probability for data in values], 
#                                           k=self.experiences_per_sampling)
#             self.sampled_batches = [random_values[i:i + batch_size] 
#                                         for i in range(0, len(random_values), self.batch_size)]



In [ ]:
import random
from collections import namedtuple, deque
import numpy as np
from numpy.random import choice
import torch

device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

class ReplayBuffer:
    """Fixed-size buffer to store experience tuples."""

    def __init__(self, action_size, buffer_size, batch_size, experiences_per_sampling, seed, compute_weights):
        """Initialize a ReplayBuffer object.
        Params
        ======
            action_size (int): dimension of each action
            buffer_size (int): maximum size of buffer
            experiences_per_sampling (int): number of experiences to sample during a sampling iteration
            batch_size (int): size of each training batch
            seed (int): random seed
        """
        self.action_size = action_size
        self.buffer_size = buffer_size
        self.batch_size = batch_size
        self.experiences_per_sampling = experiences_per_sampling
        
        self.alpha = 0.5
        self.alpha_decay_rate = 0.99
        self.beta = 0.5
        self.beta_growth_rate = 1.001
        self.seed = random.seed(seed)
        self.compute_weights = compute_weights
        self.experience_count = 0
        
        self.experience = namedtuple("Experience", 
            field_names=["state", "action", "reward", "next_state", "done"])
        self.data = namedtuple("Data", 
            field_names=["priority", "probability", "weight","index"])

        indexes = []
        datas = []
        for i in range(buffer_size):
            indexes.append(i)
            d = self.data(0,0,0,i)
            datas.append(d)
        
        self.memory = {key: self.experience for key in indexes}
        self.memory_data = {key: data for key,data in zip(indexes, datas)}
        self.sampled_batches = []
        self.current_batch = 0
        self.priorities_sum_alpha = 0
        self.priorities_max = 1
        self.weights_max = 1
    
    def update_priorities(self, tds, indices):
        for td, index in zip(tds, indices):
            N = min(self.experience_count, self.buffer_size)

            updated_priority = td[0]
            if updated_priority > self.priorities_max:
                self.priorities_max = updated_priority
            
            if self.compute_weights:
                updated_weight = ((N * updated_priority)**(-self.beta))/self.weights_max
                if updated_weight > self.weights_max:
                    self.weights_max = updated_weight
            else:
                updated_weight = 1

            old_priority = self.memory_data[index].priority
            self.priorities_sum_alpha += updated_priority**self.alpha - old_priority**self.alpha
            updated_probability = td[0]**self.alpha / self.priorities_sum_alpha
            data = self.data(updated_priority, updated_probability, updated_weight, index) 
            self.memory_data[index] = data

    def update_memory_sampling(self):
        """Randomly sample X batches of experiences from memory."""
        # X is the number of steps before updating memory
        self.current_batch = 0
        values = list(self.memory_data.values())
        random_values = random.choices(self.memory_data, 
                                       [data.probability for data in values], 
                                       k=self.experiences_per_sampling)
        self.sampled_batches = [random_values[i:i + self.batch_size] 
                                    for i in range(0, len(random_values), self.batch_size)]

    def update_parameters(self):
        self.alpha *= self.alpha_decay_rate
        self.beta *= self.beta_growth_rate
        if self.beta > 1:
            self.beta = 1
        N = min(self.experience_count, self.buffer_size)
        self.priorities_sum_alpha = 0
        sum_prob_before = 0
        for element in self.memory_data.values():
            sum_prob_before += element.probability
            self.priorities_sum_alpha += element.priority**self.alpha
        sum_prob_after = 0
        for element in self.memory_data.values():
            probability = element.priority**self.alpha / self.priorities_sum_alpha
            sum_prob_after += probability
            weight = 1
            if self.compute_weights:
                weight = ((N *  element.probability)**(-self.beta))/self.weights_max
            d = self.data(element.priority, probability, weight, element.index)
            self.memory_data[element.index] = d
        print("sum_prob before", sum_prob_before)
        print("sum_prob after : ", sum_prob_after)
    
    def add(self, state, action, reward, next_state, done):
        """Add a new experience to memory."""
        self.experience_count += 1
        index = self.experience_count % self.buffer_size

        if self.experience_count > self.buffer_size:
            temp = self.memory_data[index]
            self.priorities_sum_alpha -= temp.priority**self.alpha
            if temp.priority == self.priorities_max:
                print(temp.priority,self.priorities_max)

                self.memory_data[index].priority = 0
                # self.memory_data[index].priority._replace(0)

                self.priorities_max = max(self.memory_data.items(), key=operator.itemgetter(1)).priority
            if self.compute_weights:
                if temp.weight == self.weights_max:
                    self.memory_data[index].weight = 0
                    self.weights_max = max(self.memory_data.items(), key=operator.itemgetter(2)).weight

        priority = self.priorities_max
        weight = self.weights_max
        self.priorities_sum_alpha += priority ** self.alpha
        probability = priority ** self.alpha / self.priorities_sum_alpha
        e = self.experience(state, action, reward, next_state, done)
        self.memory[index] = e
        d = self.data(priority, probability, weight, index)
        self.memory_data[index] = d
            
    def sample(self):
        sampled_batch = self.sampled_batches[self.current_batch]
        self.current_batch += 1
        experiences = []
        weights = []
        indices = []
        
        for data in sampled_batch:
            experiences.append(self.memory.get(data.index))
            weights.append(data.weight)
            indices.append(data.index)

        states = torch.from_numpy(
            np.vstack([e.state for e in experiences if e is not None])).float()
        actions = torch.from_numpy(
            np.vstack([e.action for e in experiences if e is not None])).long()
        rewards = torch.from_numpy(
            np.vstack([e.reward for e in experiences if e is not None])).float()
        next_states = torch.from_numpy(
            np.vstack([e.next_state for e in experiences if e is not None])).float()
        dones = torch.from_numpy(
            np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float()

              # states = torch.from_numpy(
        #     np.vstack([e.state for e in experiences if e is not None])).float().to(device)
        # actions = torch.from_numpy(
        #     np.vstack([e.action for e in experiences if e is not None])).long().to(device)
        # rewards = torch.from_numpy(
        #     np.vstack([e.reward for e in experiences if e is not None])).float().to(device)
        # next_states = torch.from_numpy(
        #     np.vstack([e.next_state for e in experiences if e is not None])).float().to(device)
        # dones = torch.from_numpy(
        #     np.vstack([e.done for e in experiences if e is not None]).astype(np.uint8)).float().to(device)

        return (states, actions, rewards, next_states, dones, weights, indices)

    def __len__(self):
        """Return the current size of internal memory."""
        return len(self.memory)

### Deep Q Network

In [ ]:
class DQNP(nn.Module):
    def __init__(self, num_inputs, num_actions):
        super(DQNP, self).__init__()
        
        self.layers = nn.Sequential(
            nn.Linear(env.observation_space.shape[0], 128),
            nn.ReLU(),
            nn.Linear(128, 128),
            nn.ReLU(),
            nn.Linear(128, env.action_space.n)
        )
        
    def forward(self, x):
        return self.layers(x)
    
    def act(self, state, epsilon):
        if random.random() > epsilon:
            state   = torch.FloatTensor(state).unsqueeze(0).to(device)
            q_value = self.forward(state)
            action  = q_value.max(1)[1].data[0].item()
        else:
            action = random.randrange(env.action_space.n)
        return action

In [ ]:
UPDATE_NN_EVERY = 1        # how often to update the network

# prioritized experience replay
BATCH_SIZE = batch_size
UPDATE_MEM_EVERY = 20          # how often to update the priorities
UPDATE_MEM_PAR_EVERY = 3000     # how often to update the hyperparameters
EXPERIENCES_PER_SAMPLING = math.ceil(BATCH_SIZE * UPDATE_MEM_EVERY / UPDATE_NN_EVERY)

model = DQNP(env.observation_space.shape[0], env.action_space.n)
model = model.to(device)
    
optimizer = optim.Adam(model.parameters())

# replay_buffer = ReplayBuffer(1000)
replay_buffer = ReplayBuffer(env.action_space.n,1000,batch_size, EXPERIENCES_PER_SAMPLING,seed,False)

### Compute Loss

In [ ]:
def compute_td_loss():
    state, action, reward, next_state, done, weights, indices  = replay_buffer.sample()

    # print(state.shape,action.shape)
    # print(env.action_space.n)
    # print(EXPERIENCES_PER_SAMPLING)
    # print(action.shape)
    # print(action.unsqueeze(1))

    state      = torch.FloatTensor(np.float32(state)).to(device)
    next_state = torch.FloatTensor(np.float32(next_state)).to(device)
    action     = torch.LongTensor(action).to(device)
    reward     = torch.FloatTensor(reward).to(device)
    done       = torch.FloatTensor(done).to(device)

    q_values      = model(state)
    next_q_values = model(next_state)

    q_value          = q_values.gather(1, action).squeeze(1)
    # q_value          = q_values.gather(1, action.unsqueeze(1)).squeeze(1)

    next_q_value     = next_q_values.max(1)[0]
    expected_q_value = reward + gamma * next_q_value  * (1 - done)
    
    loss = (q_value - expected_q_value.data).pow(2).mean()
        
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()
    
    return loss

### training

In [ ]:


losses = []
all_rewards = []
episode_reward = 0

state = env.reset()

replay_buffer.update_memory_sampling()
# replay_buffer.update_parameters()

for frame_idx in range(num_frames ):
    epsilon = epsilon_by_frame(frame_idx+1)
    action = model.act(state, epsilon)
    
    next_state, reward, done, _ = env.step(action)
    print(frame_idx)
    replay_buffer.add(state, action, reward, next_state, done)

    if frame_idx % UPDATE_MEM_PAR_EVERY == 0:
            replay_buffer.update_parameters()
    if frame_idx % UPDATE_NN_EVERY == 0:
        # If enough samples are available in memory, get random subset and learn
        # if replay_buffer.experience_count > EXPERIENCES_PER_SAMPLING:
        if len(replay_buffer) > batch_size:
            loss = compute_td_loss()
            losses.append(loss.item())
                # sampling = replay_buffer.sample()
            # self.learn(sampling, GAMMA)
    if frame_idx % UPDATE_MEM_EVERY == 0:
        replay_buffer.update_memory_sampling()
    
    state = next_state
    episode_reward += reward
    
    if done:
        state = env.reset()
        all_rewards.append(episode_reward)
        episode_reward = 0

    # if len(replay_buffer) > batch_size:
    #     loss = compute_td_loss()
    #     losses.append(loss.item())



0
sum_prob before 1.0
sum_prob after :  1.0


TypeError: ignored

In [ ]:
from IPython.display import clear_output
import matplotlib.pyplot as plt
%matplotlib inline

plt.title('loss')
plt.plot(losses)
plt.show()

plt.title('reward')
plt.plot(all_rewards)
plt.show()



## Generate Video

In [ ]:
"""
Utility functions to enable video recording of gym environment 
and displaying it.
To enable video, just do "env = wrap_env(env)""
"""

def show_video():
    mp4list = glob.glob('video/*.mp4')
    if len(mp4list) > 0:
        mp4 = mp4list[0]
        video = io.open(mp4, 'r+b').read()
        encoded = base64.b64encode(video)
        ipythondisplay.display(HTML(data='''<video alt="test" autoplay
                loop controls style="height: 400px;">
                <source src="data:video/mp4;base64,{0}" type="video/mp4" />
             </video>'''.format(encoded.decode('ascii'))))
    else:
        print("Could not find video")
    

def wrap_env(env):
    display = Display(visible=0, size=(1400, 900))
    display.start()
    env = Monitor(env, './video/dqnp', force=True)
    return env

In [ ]:
# env = wrap_env(gym.make("BreakoutDeterministic-v4"))
env = wrap_env(gym.make("CartPole-v0"))



observation = env.reset()

while True:
  
    # env.render()
    
    #your agent goes here
    action = model.act(torch.tensor(observation, dtype=torch.float), 0)
         
    observation, reward, done, info = env.step(action) 
        
    if done: 
        break
            
env.close()
show_video()